### Disclaimer
Please note, the Vantage Functions via SQLAlchemy feature is a preview/beta code release with limited functionality (the “Code”). As such, you acknowledge that the Code is experimental in nature and that the Code is provided “AS IS” and may not be functional on any machine or in any environment. TERADATA DISCLAIMS ALL WARRANTIES RELATING TO THE CODE, EXPRESS OR IMPLIED, INCLUDING, BUT NOT LIMITED TO, ANY WARRANTIES AGAINST INFRINGEMENT OF THIRD-PARTY RIGHTS, MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE.

TERADATA SHALL NOT BE RESPONSIBLE OR LIABLE WITH RESPECT TO ANY SUBJECT MATTER OF THE CODE UNDER ANY CONTRACT, NEGLIGENCE, STRICT LIABILITY OR OTHER THEORY 
    (A) FOR LOSS OR INACCURACY OF DATA OR COST OF PROCUREMENT OF SUBSTITUTE GOODS, SERVICES OR TECHNOLOGY, OR 
    (B) FOR ANY INDIRECT, INCIDENTAL OR CONSEQUENTIAL DAMAGES INCLUDING, BUT NOT LIMITED TO LOSS OF REVENUES AND LOSS OF PROFITS. TERADATA SHALL NOT BE RESPONSIBLE FOR ANY MATTER BEYOND ITS REASONABLE CONTROL.

Notwithstanding anything to the contrary: 
    (a) Teradata will have no obligation of any kind with respect to any Code-related comments, suggestions, design changes or improvements that you elect to provide to Teradata in either verbal or written form (collectively, “Feedback”), and 
    (b) Teradata and its affiliates are hereby free to use any ideas, concepts, know-how or techniques, in whole or in part, contained in Feedback: 
        (i) for any purpose whatsoever, including developing, manufacturing, and/or marketing products and/or services incorporating Feedback in whole or in part, and 
        (ii) without any restrictions or limitations, including requiring the payment of any license fees, royalties, or other consideration. 

In [1]:
# In this notebook we shall cover examples for following Regular Expression Functions:
# SQL Documentation: https://docs.teradata.com/reader/756LNiPSFdY~4JcCCcR5Cw/c2fX4dzxCcDJFKqXbyQtTA

# TD Supported Window Aggregate fucntions:
# 1. avg
# 2. count
# 3. corr
# 4. max
# 5. min
# 6. stddev_pop
# 7. stddev_samp
# 8. sum
# 9. var_pop
# 10. var_samp
# 11. REGR_AVGX
# 12. REGR_AVGY
# 13. REGR_INTERCEPT
# 14. REGR_R2
# 15. REGR_SLOPE
# 16. REGR_SXX
# 17. REGR_SXY
# 18. REGR_SYY
# 19. cume_dist
# 20. dense_rank
# 21. first_value
# 22. last_value
# 23. lag (Without Ignore NULLs)
# 24. lead (Without Ignore NULLs)
# 25. mdiff 
# 26. CSUM
# 27. MAVG
# 28. median --> with drop_columns=True
# 29. mlinreg
# 30. MSUM
# 31. percent_rank
# 32. percentile_cont --> with drop_columns=True
# 33. percentile_disc --> with drop_columns=True
# 34. Quantile
# 35. RANK --> Without "with ties" and "RESET WHEN"
# 36. RANK(Teradata)
# 37. row_number() --> Without "RESET WHEN"
# 38. covar_samp
# 39. covar_pop

In [2]:
# Get the connection to the Vantage using create_context()
import getpass
from teradataml import *
td_context = create_context(host=getpass.getpass("Hostname: "), username=getpass.getpass("Username: "), password=getpass.getpass("Passowrd: "))
# Load the example dataset.
load_example_data("GLM", ["admissions_train"])

Hostname: ········
Username: ········
Passowrd: ········


In [3]:
# Create the DataFrame on 'admissions_train' table
admissions_train = DataFrame("admissions_train")
admissions_train

   masters   gpa     stats programming  admitted
id                                              
5       no  3.44    Novice      Novice         0
34     yes  3.85  Advanced    Beginner         0
13      no  4.00  Advanced      Novice         1
40     yes  3.95    Novice    Beginner         0
22     yes  3.46    Novice    Beginner         0
19     yes  1.98  Advanced    Advanced         0
36      no  3.00  Advanced      Novice         0
15     yes  4.00  Advanced    Advanced         1
7      yes  2.33    Novice      Novice         1
17      no  3.83  Advanced    Advanced         1

In [4]:
def print_variables(df, columns=None):
    print("Equivalent SQL: {}".format(df.show_query()))
    print("\n")
    print(" ************************* DataFrame ********************* ")
    print(df)
    print("\n\n")
    print(" ************************* DataFrame.dtypes ********************* ")
    print(df.dtypes)
    print("\n\n")
    if columns is not None:
        if isinstance(columns, str):
            columns = [columns]
        for col in columns:
            coltype = df.__getattr__(col).type
            if isinstance(coltype, sqlalchemy.sql.sqltypes.NullType):
                coltype = "NullType"
            print(" '{}' Column Type: {}".format(col, coltype))

# Using SQLAlchemy ClasueElements for WindowAggregates in teradataml DataFrame.assign()

In [5]:
# Before we move on with examples, one should read below just to understand how teradataml DataFrame and 
# it's columns are used to create a SQLAlchemy ClauseElement/Expression.

# Often in below examples one would see something like this: 'admissions_train.admitted.expression'
# Here in the above expression,
#    'admissions_train' is 'teradataml DataFrame'
#    'admitted' is 'column name' in teradataml DataFrame 'admissions_train'
#    Thus, 
#        'admissions_train.admitted' together forms a ColumnExpression.
#    expression allows us to use teradata ColumnExpression to be treated as SQLAlchemy Expression.
#    Thus,
#        'admissions_train.admitted.expression' gives us an expression that can be used with SQLAlchemy clauseElements.

## Using SQLAlchemy within_group expression

### percentile_disc and percentile_cont function

In [6]:
# Returns an interpolated value that falls within its value_expression with respect to its sort specification.
# SQL Syntax:
#        function_name(value_expression) WITHIN GROUP(ORDER BY order_by_expression)
#        where,
#                value_expression - a column expression
#                order_by_expression - a numeric column expression for sorting
#                                      ordering can be done in Ascending or Descending order with Nulls First or Nulls Last
#
# SQLAlchmey Syntax:
#        func.function_name(value_expression).within_group(order_by_expression)
#        where,
#                value_expression - a SQLAlchemy column expression
#                order_by_expression - a SQLAlchemy column expression of numeric type for sorting
#                                      ordering can be done in Ascending or Descending order with Nulls First or Nulls Last

In [7]:
# Import required modules
from sqlalchemy import func

In [8]:
# Execute percentile_cont function using WITHIN GROUP with desceniding odering done on gpa column 
pc = func.percentile_cont(0.5).within_group(
        admissions_train.gpa.expression.desc()
    )
type(pc)

sqlalchemy.sql.elements.WithinGroup

In [9]:
pc_df = admissions_train.assign(True, interpolated_value=pc)
print_variables(pc_df, "interpolated_value")

Equivalent SQL: select percentile_cont(0.5) WITHIN GROUP (ORDER BY gpa DESC) AS interpolated_value from "admissions_train"


 ************************* DataFrame ********************* 
   interpolated_value
0                3.69



 ************************* DataFrame.dtypes ********************* 
interpolated_value    float



 'interpolated_value' Column Type: FLOAT


In [10]:
# Execute percentile_cont function using WITHIN GROUP with ascending odering done on gpa column and with NULL FIRST
df = admissions_train.assign(True, interpolated_value=func.percentile_cont(1).within_group(
    admissions_train.gpa.expression.asc().nullsfirst()))
print_variables(df, "interpolated_value")

Equivalent SQL: select percentile_cont(1) WITHIN GROUP (ORDER BY gpa ASC NULLS FIRST) AS interpolated_value from "admissions_train"


 ************************* DataFrame ********************* 
   interpolated_value
0                 4.0



 ************************* DataFrame.dtypes ********************* 
interpolated_value    float



 'interpolated_value' Column Type: FLOAT


In [11]:
# Execute percentile_disc function using WITHIN GROUP with odering done on gpa column and with NULLS LAST
df = admissions_train.assign(True, interpolated_value=func.percentile_disc(1).within_group(
    admissions_train.gpa.expression.nullslast()))
print_variables(df, "interpolated_value")

Equivalent SQL: select percentile_disc(1) WITHIN GROUP (ORDER BY gpa NULLS LAST) AS interpolated_value from "admissions_train"


 ************************* DataFrame ********************* 
   interpolated_value
0                 4.0



 ************************* DataFrame.dtypes ********************* 
interpolated_value    float



 'interpolated_value' Column Type: FLOAT


## Windowed Aggreates using OVER 

In [12]:
# Windowed aggregation offered by Vantage supportes various clauses and can be used 
# in teradataml with the help of SQLAlchemy.
# Let's take a look at few examples on how can we perform windowed aggregation with 
# the help of SQLAlchemy and teradataml's DataFrame.assign() API.


# Column ordering in SQLAlchemy:
# To specify ascending or descending ordering, one can use asc() or desc() methods on SQLAlchemy column.
# One can specify nulls first or nulls last as well for odering. 
# Here is how you do it:
#        Descending oder: admissions_train.id.expression.desc() 
#        Ascending oder: admissions_train.id.expression.asc() 
#        With NULLS FIRST - df.column_name.expression.nullsfirst()
#        With NULLS LAST - df.column_name.expression.nullslast()
#        Ascending Order with NULLS LAST - df.column_name.expression.asc().nullslast()


# SQLAlchemy offers a method over(...), that can be operated on a  SQLAlchemy column.
# Syntax:
#        over(partition_by = partition_expression, order_by = order_expression, rows = (p, f))
#        where,
#                partition_expression - column expression to partition the data on. 
#                                       For example,
#                                               df.column_name.expression
#                order_expression - column expression to sort the data. 
#                                   Sorting can be done either in Ascending or Descending order with NULLS FIRST or NULLS LAST.
#                                   For example,
#                                           Default sorting - df.column_name.expression
#                                           Ascending Order - df.column_name.expression.asc()
#                                           Descending Order - df.column_name.expression.desc()
#                                           With NULLS FIRST - df.column_name.expression.nullsfirst()
#                                           With NULLS LAST - df.column_name.expression.nullslast()
#                                           Ascending Order with NULLS LAST - df.column_name.expression.asc().nullslast()
#                rows - Generates the syntax for 'ROWS BETWEEN'
#                        To perform windowed aggregate function over a window using ROWS and ROWS BETWEEN one must use 
#                        argument "rows" in over which accepts a tuple (p, f).
#                        Where,
#                                p and f can accept Negative Integer Value, Positive Integer Value, 0 or None.
#                                Each value passed to p and f have different meaning or results in different syntax.
#                                SQL syntax will be generated based on these values:
#                                        1. Negative Value --> Indicates a preceding value
#                                        2. Positive Value --> Indiacates a following value
#                                        3. 0 --> For Current row
#                                        4. None --> Unbounded value.
#

### CSUM Function

In [13]:
# Returns the cumulative (or running) sum of a value expression for each row in a partition, 
# assuming the rows in the partition are sorted by the sort_expression list.
# Syntax:
#        CSUM(value_expression, sort_expression ASC/DESC)
#        where,
#                value_expression - a SQLAlchemy column expression
#                order_by_expression - a SQLAlchemy column expression for sorting
#                                      ordering can be done in Ascending or Descending order

# To specify ascending or descending ordering, one can use asc() or desc() methods on SQLAlchemy column.
# For example,
#        Descending oder: admissions_train.id.expression.desc() 
#        Ascending oder: admissions_train.id.expression.asc() 

In [14]:
# Calculate running total of gpa.
csum_ = func.csum(admissions_train.gpa.expression, admissions_train.id.expression.desc())
type(csum_)

sqlalchemy.sql.functions.Function

In [15]:
csum_gpa_df = admissions_train.assign(csum_gpa_=csum_)
print_variables(csum_gpa_df, "csum_gpa_")

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, csum(gpa, id DESC) AS csum_gpa_ from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  csum_gpa_
id                                                         
38     yes  2.65  Advanced    Beginner         1      10.35
36      no  3.00  Advanced      Novice         0      16.87
35      no  3.68    Novice    Beginner         1      20.55
34     yes  3.85  Advanced    Beginner         0      24.40
32     yes  3.46  Advanced    Beginner         0      31.41
31     yes  3.50  Advanced    Beginner         1      34.91
33      no  3.55    Novice      Novice         1      27.95
37      no  3.52    Novice      Novice         1      13.87
39     yes  3.75  Advanced    Beginner         0       7.70
40     yes  3.95    Novice    Beginner         0       3.95



 ************************* D

### CUME_DIST function

In [16]:
# Calculates the cumulative distribution of a value in a group of values.
# SQL Syntax:
#        CUME_DIST() OVER (partition_by_expression, sort_expression)
# SQLAlchemy Syntax:
#        func.CUME_DIST().over(partition_by = partition_by_expression, order_by = sort_expression)
#        where,
#                partition_by_expression - a SQLAlchemy column expression
#                order_by_expression - a SQLAlchemy column expression for sorting
#                                      ordering can be done in Ascending or Descending order with NULLS FIRST or NULLS LAST

In [17]:
# Calculate cumulative distribution by stats.
cum_dist_ = func.Cume_dist().over(order_by=admissions_train.stats.expression.desc())
type(cum_dist_)

sqlalchemy.sql.elements.Over

In [18]:
cum_dist_by_stats_df = admissions_train.assign(cum_dist_by_stats=cum_dist_)
print_variables(cum_dist_by_stats_df, "cum_dist_by_stats")

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, cume_dist() OVER (ORDER BY stats DESC) AS cum_dist_by_stats from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa   stats programming  admitted  cum_dist_by_stats
id                                                               
29     yes  4.00  Novice    Beginner         0              0.275
35      no  3.68  Novice    Beginner         1              0.275
37      no  3.52  Novice      Novice         1              0.275
40     yes  3.95  Novice    Beginner         0              0.275
12      no  3.65  Novice      Novice         1              0.275
22     yes  3.46  Novice    Beginner         0              0.275
7      yes  2.33  Novice      Novice         1              0.275
3       no  3.70  Novice    Beginner         1              0.275
33      no  3.55  Novice      Novice         1              0.2

In [19]:
# Calculate cumulative distribution by id, partitioned over stats
cum_dist_ = func.Cume_dist().over(partition_by=admissions_train.stats.expression, order_by=admissions_train.id.expression.desc())
type(cum_dist_)

sqlalchemy.sql.elements.Over

In [20]:
cum_dist_by_stats_df = admissions_train.assign(cum_dist_by_stats=cum_dist_)
print_variables(cum_dist_by_stats_df, "cum_dist_by_stats")

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, cume_dist() OVER (PARTITION BY stats ORDER BY id DESC) AS cum_dist_by_stats from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  cum_dist_by_stats
id                                                                 
4      yes  3.50  Beginner      Novice         1           0.600000
1      yes  3.95  Beginner    Beginner         0           1.000000
39     yes  3.75  Advanced    Beginner         0           0.041667
38     yes  2.65  Advanced    Beginner         1           0.083333
34     yes  3.85  Advanced    Beginner         0           0.166667
32     yes  3.46  Advanced    Beginner         0           0.208333
31     yes  3.50  Advanced    Beginner         1           0.250000
30     yes  3.79  Advanced      Novice         0           0.291667
28      no  3.93  Advanced 

### DENSE_RANK fucntion

In [21]:
# Returns an ordered ranking of rows based on the value_expression in the ORDER BY clause.
# SQL Syntax:
#        DENSE_RANK() OVER (PARTITION BY partition_by_expression, ORDER BY sort_expression)
#
# SQLAlchemy Syntax:
#        func.DENSE_RANK().over(partition_by = partition_by_expression, order_by = sort_expression)
#        where,
#                partition_by_expression - a SQLAlchemy column expression
#                order_by_expression - a SQLAlchemy column expression for sorting
#                                      ordering can be done in Ascending or Descending order with NULLS FIRST or NULLS LAST

In [22]:
# Get the ordered ranking based on stats.
dense_rank_ = func.DENSE_RANK().over(order_by=admissions_train.stats.expression.desc())
type(dense_rank_)

sqlalchemy.sql.elements.Over

In [23]:
dense_rank_by_stats_df = admissions_train.assign(dense_rank_by_stats=dense_rank_)
print_variables(dense_rank_by_stats_df, "dense_rank_by_stats")

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, dense_rank() OVER (ORDER BY stats DESC) AS dense_rank_by_stats from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa   stats programming  admitted  dense_rank_by_stats
id                                                                 
33      no  3.55  Novice      Novice         1                    1
12      no  3.65  Novice      Novice         1                    1
5       no  3.44  Novice      Novice         0                    1
40     yes  3.95  Novice    Beginner         0                    1
3       no  3.70  Novice    Beginner         1                    1
21      no  3.87  Novice    Beginner         1                    1
29     yes  4.00  Novice    Beginner         0                    1
22     yes  3.46  Novice    Beginner         0                    1
35      no  3.68  Novice    Beginner    

In [24]:
# Calculate cumulative distribution by id, partitioned over stats
dense_rank_ = func.dense_rank().over(partition_by=admissions_train.stats.expression, order_by=admissions_train.id.expression.desc())
type(dense_rank_)

sqlalchemy.sql.elements.Over

In [25]:
dense_rank_by_stats_df = admissions_train.assign(dense_rank_by_stats=dense_rank_)
print_variables(dense_rank_by_stats_df, "dense_rank_by_stats")

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, dense_rank() OVER (PARTITION BY stats ORDER BY id DESC) AS dense_rank_by_stats from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  dense_rank_by_stats
id                                                                   
4      yes  3.50  Beginner      Novice         1                    3
1      yes  3.95  Beginner    Beginner         0                    5
39     yes  3.75  Advanced    Beginner         0                    1
38     yes  2.65  Advanced    Beginner         1                    2
34     yes  3.85  Advanced    Beginner         0                    4
32     yes  3.46  Advanced    Beginner         0                    5
31     yes  3.50  Advanced    Beginner         1                    6
30     yes  3.79  Advanced      Novice         0                    7
28  

### First Value Function

In [26]:
# Returns the first value in an ordered set of values.
# SQL Syntax:
#        FIRST_VALUE(value_expression) OVER (PARTITION BY partition_by_expression, ORDER BY sort_expression, ROWS BETWEEN ... )
#
# SQLAlchemy Syntax:
#        func.FIRST_VALUE(value_expression).over(partition_by = partition_by_expression, order_by = sort_expression, rows=(p, f))
#        where,
#                value_expression - a SQLAlchemy column expression on which function to be executed.
#                partition_by_expression - a SQLAlchemy column expression
#                order_by_expression - a SQLAlchemy column expression for sorting
#                                      ordering can be done in Ascending or Descending order with NULLS FIRST or NULLS LAST
#                rows_ = Define a window

In [27]:
# Example returns by id first gpa in the moving average group.
FIRST_VALUE_ = func.FIRST_VALUE(admissions_train.gpa.expression).over(order_by=admissions_train.id.expression, rows=(-3, 1))
type(FIRST_VALUE_)

sqlalchemy.sql.elements.Over

In [28]:
fv_gpa_df = admissions_train.assign(FIRST_VALUE_gpa=FIRST_VALUE_)
print_variables(fv_gpa_df, "FIRST_VALUE_gpa")

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, FIRST_VALUE(gpa) OVER (ORDER BY id ROWS BETWEEN 3 PRECEDING AND 1 FOLLOWING) AS "FIRST_VALUE_gpa" from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  FIRST_VALUE_gpa
id                                                               
3       no  3.70    Novice    Beginner         1             3.95
5       no  3.44    Novice      Novice         0             3.76
6      yes  3.50  Beginner    Advanced         1             3.70
7      yes  2.33    Novice      Novice         1             3.50
9       no  3.82  Advanced    Advanced         1             3.50
10      no  3.71  Advanced    Advanced         1             2.33
8       no  3.60  Beginner    Advanced         1             3.44
4      yes  3.50  Beginner      Novice         1             3.95
2      yes  3.76  Beginne

### Last Value Function

In [29]:
# Returns the last value in an ordered set of values.
# SQL Syntax:
#        LAST_VALUE(value_expression) OVER (PARTITION BY partition_by_expression, ORDER BY sort_expression, ROWS BETWEEN ... )
#
# SQLAlchemy Syntax:
#        func.LAST_VALUE(value_expression).over(partition_by = partition_by_expression, order_by = sort_expression, rows=(p, f))
#        where,
#                value_expression - a SQLAlchemy column expression on which function to be executed.
#                partition_by_expression - a SQLAlchemy column expression
#                order_by_expression - a SQLAlchemy column expression for sorting
#                                      ordering can be done in Ascending or Descending order with NULLS FIRST or NULLS LAST
#                rows_ = Define a window

In [30]:
# Example returns by id last gpa in the moving average group.
last_value_ = func.last_value(admissions_train.gpa.expression).over(order_by=admissions_train.id.expression, rows=(-3, 1))
type(last_value_)

sqlalchemy.sql.elements.Over

In [31]:
lv_gpa_df = admissions_train.assign(last_value_gpa=last_value_)
print_variables(lv_gpa_df, "last_value_gpa")

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, last_value(gpa) OVER (ORDER BY id ROWS BETWEEN 3 PRECEDING AND 1 FOLLOWING) AS last_value_gpa from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  last_value_gpa
id                                                              
3       no  3.70    Novice    Beginner         1            3.50
5       no  3.44    Novice      Novice         0            3.50
6      yes  3.50  Beginner    Advanced         1            2.33
7      yes  2.33    Novice      Novice         1            3.60
9       no  3.82  Advanced    Advanced         1            3.71
10      no  3.71  Advanced    Advanced         1            3.13
8       no  3.60  Beginner    Advanced         1            3.82
4      yes  3.50  Beginner      Novice         1            3.44
2      yes  3.76  Beginner    Beginner 

### LEAD function

In [32]:
# Example returns by id last gpa in the moving average group.
lead_value_ = func.lead(admissions_train.gpa.expression, 3, 
                        admissions_train.admitted.expression).over(order_by=admissions_train.id.expression)
type(lead_value_)

sqlalchemy.sql.elements.Over

In [33]:
lead_gpa_df = admissions_train.assign(lead_value_col=lead_value_)
print_variables(lead_gpa_df, "lead_value_col")

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, lead(gpa, 3, admitted) OVER (ORDER BY id) AS lead_value_col from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  lead_value_col
id                                                              
3       no  3.70    Novice    Beginner         1            3.50
5       no  3.44    Novice      Novice         0            3.60
6      yes  3.50  Beginner    Advanced         1            3.82
7      yes  2.33    Novice      Novice         1            3.71
9       no  3.82  Advanced    Advanced         1            3.65
10      no  3.71  Advanced    Advanced         1            4.00
8       no  3.60  Beginner    Advanced         1            3.13
4      yes  3.50  Beginner      Novice         1            2.33
2      yes  3.76  Beginner    Beginner         0            3.44
1      y

### LAG function

In [34]:
# Example returns by id last gpa in the moving average group.
lag_value_ = func.LAG(admissions_train.gpa.expression, 3, 
                        admissions_train.admitted.expression).over(order_by=admissions_train.id.expression)
type(lag_value_)

sqlalchemy.sql.elements.Over

In [35]:
lag_gpa_df = admissions_train.assign(lag_value_col=lag_value_)
print_variables(lag_gpa_df, "lag_value_col")

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, LAG(gpa, 3, admitted) OVER (ORDER BY id) AS lag_value_col from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  lag_value_col
id                                                             
3       no  3.70    Novice    Beginner         1           1.00
5       no  3.44    Novice      Novice         0           3.76
6      yes  3.50  Beginner    Advanced         1           3.70
7      yes  2.33    Novice      Novice         1           3.50
9       no  3.82  Advanced    Advanced         1           3.50
10      no  3.71  Advanced    Advanced         1           2.33
8       no  3.60  Beginner    Advanced         1           3.44
4      yes  3.50  Beginner      Novice         1           3.95
2      yes  3.76  Beginner    Beginner         0           0.00
1      yes  3.95  Beg

### MAVG Function

In [36]:
# Computes the moving average of a value expression for each row in a partition using the specified value expression 
# for the current row and the preceding width-1 rows.

In [37]:
mavg_ = func.mavg(admissions_train.gpa.expression, 3, admissions_train.id.expression)
type(mavg_)

sqlalchemy.sql.functions.Function

In [38]:
mavg_df = admissions_train.assign(mavg_col=mavg_)
print_variables(mavg_df, "mavg_col")

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, mavg(gpa, 3, id) AS mavg_col from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  mavg_col
id                                                        
3       no  3.70    Novice    Beginner         1  3.803333
5       no  3.44    Novice      Novice         0  3.546667
6      yes  3.50  Beginner    Advanced         1  3.480000
7      yes  2.33    Novice      Novice         1  3.090000
9       no  3.82  Advanced    Advanced         1  3.250000
10      no  3.71  Advanced    Advanced         1  3.710000
8       no  3.60  Beginner    Advanced         1  3.143333
4      yes  3.50  Beginner      Novice         1  3.653333
2      yes  3.76  Beginner    Beginner         0  3.855000
1      yes  3.95  Beginner    Beginner         0  3.950000



 ************************* DataFrame.dtypes

### MDIFF Function

In [39]:
# Returns the moving difference between the specified value expression for the current row and the preceding width rows 
# for each row in the partition.

In [40]:
mdiff_ = func.mdiff(admissions_train.gpa.expression, 3, admissions_train.id.expression)
type(mdiff_)

sqlalchemy.sql.functions.Function

In [41]:
mdiff_df = admissions_train.assign(mdiff_col=mdiff_)
print_variables(mdiff_df, "mdiff_col")

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, mdiff(gpa, 3, id) AS mdiff_col from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  mdiff_col
id                                                         
3       no  3.70    Novice    Beginner         1        NaN
5       no  3.44    Novice      Novice         0      -0.32
6      yes  3.50  Beginner    Advanced         1      -0.20
7      yes  2.33    Novice      Novice         1      -1.17
9       no  3.82  Advanced    Advanced         1       0.32
10      no  3.71  Advanced    Advanced         1       1.38
8       no  3.60  Beginner    Advanced         1       0.16
4      yes  3.50  Beginner      Novice         1      -0.45
2      yes  3.76  Beginner    Beginner         0        NaN
1      yes  3.95  Beginner    Beginner         0        NaN



 ************************* Da

### Median function

In [42]:
median_ = func.median(admissions_train.gpa.expression)
type(median_)

sqlalchemy.sql.functions.Function

In [43]:
median_df = admissions_train.assign(True, median_col=median_)
print_variables(median_df, "median_col")

Equivalent SQL: select median(gpa) AS median_col from "admissions_train"


 ************************* DataFrame ********************* 
   median_col
0        3.69



 ************************* DataFrame.dtypes ********************* 
median_col    float



 'median_col' Column Type: FLOAT


### MLINREG function

In [44]:
# Returns a predicted value for an expression based on a least squares moving linear regression of the previous 
# width -1 (based on sort_expression) column values.

In [45]:
MLINREG_ = func.MLINREG(admissions_train.gpa.expression, 3, admissions_train.id.expression)
type(MLINREG_)

sqlalchemy.sql.functions.Function

In [46]:
MLINREG_df = admissions_train.assign(MLINREG_col=MLINREG_)
print_variables(MLINREG_df, "MLINREG_col")

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, MLINREG(gpa, 3, id) AS "MLINREG_col" from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  MLINREG_col
id                                                           
3       no  3.70    Novice    Beginner         1         3.57
5       no  3.44    Novice      Novice         0         3.30
6      yes  3.50  Beginner    Advanced         1         3.38
7      yes  2.33    Novice      Novice         1         3.56
9       no  3.82  Advanced    Advanced         1         4.87
10      no  3.71  Advanced    Advanced         1         4.04
8       no  3.60  Beginner    Advanced         1         1.16
4      yes  3.50  Beginner      Novice         1         3.64
2      yes  3.76  Beginner    Beginner         0          NaN
1      yes  3.95  Beginner    Beginner         0          NaN




### MSUM function

In [47]:
# Computes the moving sum specified by a value expression for the current row and the preceding n-1 rows. 
# This function is very similar to the MAVG function.

In [48]:
msum_ = func.msum(admissions_train.gpa.expression, 3, admissions_train.id.expression)
type(msum_)

sqlalchemy.sql.functions.Function

In [49]:
msum_df = admissions_train.assign(msum_col=msum_)
print_variables(msum_df, "msum_col")

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, msum(gpa, 3, id) AS msum_col from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  msum_col
id                                                        
3       no  3.70    Novice    Beginner         1     11.41
5       no  3.44    Novice      Novice         0     10.64
6      yes  3.50  Beginner    Advanced         1     10.44
7      yes  2.33    Novice      Novice         1      9.27
9       no  3.82  Advanced    Advanced         1      9.75
10      no  3.71  Advanced    Advanced         1     11.13
8       no  3.60  Beginner    Advanced         1      9.43
4      yes  3.50  Beginner      Novice         1     10.96
2      yes  3.76  Beginner    Beginner         0      7.71
1      yes  3.95  Beginner    Beginner         0      3.95



 ************************* DataFrame.dtypes

### PERCENT_RANK function

In [50]:
# Returns the relative rank of rows for a value_expression.

In [51]:
# Get the ordered ranking based on stats.
percent_rank_ = func.percent_rank().over(order_by=admissions_train.id.expression.desc())
type(percent_rank_)

sqlalchemy.sql.elements.Over

In [52]:
percent_rank_df = admissions_train.assign(percent_rank_col=percent_rank_)
print_variables(percent_rank_df, "percent_rank_col")

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, percent_rank() OVER (ORDER BY id DESC) AS percent_rank_col from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  percent_rank_col
id                                                                
38     yes  2.65  Advanced    Beginner         1          0.051282
36      no  3.00  Advanced      Novice         0          0.102564
35      no  3.68    Novice    Beginner         1          0.128205
34     yes  3.85  Advanced    Beginner         0          0.153846
32     yes  3.46  Advanced    Beginner         0          0.205128
31     yes  3.50  Advanced    Beginner         1          0.230769
33      no  3.55    Novice      Novice         1          0.179487
37      no  3.52    Novice      Novice         1          0.076923
39     yes  3.75  Advanced    Beginner         0      

In [53]:
# Calculate cumulative distribution by id, partitioned over stats
percent_rank_ = func.percent_rank().over(partition_by=admissions_train.stats.expression, order_by=admissions_train.id.expression.desc())
type(percent_rank_)

sqlalchemy.sql.elements.Over

In [54]:
percent_rank_df = admissions_train.assign(percent_rank_col=percent_rank_)
print_variables(percent_rank_df, "percent_rank_col")

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, percent_rank() OVER (PARTITION BY stats ORDER BY id DESC) AS percent_rank_col from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  percent_rank_col
id                                                                
4      yes  3.50  Beginner      Novice         1          0.500000
1      yes  3.95  Beginner    Beginner         0          1.000000
39     yes  3.75  Advanced    Beginner         0          0.000000
38     yes  2.65  Advanced    Beginner         1          0.043478
34     yes  3.85  Advanced    Beginner         0          0.130435
32     yes  3.46  Advanced    Beginner         0          0.173913
31     yes  3.50  Advanced    Beginner         1          0.217391
30     yes  3.79  Advanced      Novice         0          0.260870
28      no  3.93  Advanced    Advan

### Quantile Function

In [55]:
quantile_ = func.quantile(10, admissions_train.gpa.expression)
type(quantile_)

sqlalchemy.sql.functions.Function

In [56]:
quantile_df = admissions_train.assign(quantile_col=quantile_)
print_variables(quantile_df, "quantile_col")

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, quantile(10, gpa) AS quantile_col from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  quantile_col
id                                                            
7      yes  2.33    Novice      Novice         1             0
36      no  3.00  Advanced      Novice         0             1
11      no  3.13  Advanced    Advanced         1             1
5       no  3.44    Novice      Novice         0             1
32     yes  3.46  Advanced    Beginner         0             2
22     yes  3.46    Novice    Beginner         0             2
14     yes  3.45  Advanced    Advanced         0             1
38     yes  2.65  Advanced    Beginner         1             0
19     yes  1.98  Advanced    Advanced         0             0
24      no  1.87  Advanced      Novice         1        

### RANK function

In [57]:
# Get the ordered ranking based on stats.
rank_ = func.rank().over(order_by=admissions_train.id.expression.desc())
type(rank_)

sqlalchemy.sql.elements.Over

In [58]:
rank_df = admissions_train.assign(rank_col=rank_)
print_variables(rank_df, "rank_col")

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, rank() OVER (ORDER BY id DESC) AS rank_col from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  rank_col
id                                                        
38     yes  2.65  Advanced    Beginner         1         3
36      no  3.00  Advanced      Novice         0         5
35      no  3.68    Novice    Beginner         1         6
34     yes  3.85  Advanced    Beginner         0         7
32     yes  3.46  Advanced    Beginner         0         9
31     yes  3.50  Advanced    Beginner         1        10
33      no  3.55    Novice      Novice         1         8
37      no  3.52    Novice      Novice         1         4
39     yes  3.75  Advanced    Beginner         0         2
40     yes  3.95    Novice    Beginner         0         1



 ************************* Da

In [59]:
# Calculate cumulative distribution by id, partitioned over stats
rank_ = func.rank().over(partition_by=admissions_train.stats.expression, order_by=admissions_train.id.expression.desc())
type(rank_)

sqlalchemy.sql.elements.Over

In [60]:
rank_df = admissions_train.assign(rank_col=rank_)
print_variables(rank_df, "rank_col")

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, rank() OVER (PARTITION BY stats ORDER BY id DESC) AS rank_col from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  rank_col
id                                                        
36      no  3.00  Advanced      Novice         0         3
32     yes  3.46  Advanced    Beginner         0         5
31     yes  3.50  Advanced    Beginner         1         6
30     yes  3.79  Advanced      Novice         0         7
27     yes  3.96  Advanced    Advanced         0         9
26     yes  3.57  Advanced    Advanced         1        10
8       no  3.60  Beginner    Advanced         1         1
6      yes  3.50  Beginner    Advanced         1         2
4      yes  3.50  Beginner      Novice         1         3
2      yes  3.76  Beginner    Beginner         0         4



 *********

### ROW_NUMBER function

In [61]:
# Get the ordered ranking based on stats.
row_num_ = func.row_number().over(order_by=admissions_train.id.expression.desc())
type(row_num_)

sqlalchemy.sql.elements.Over

In [62]:
row_num_df = admissions_train.assign(row_num_col=row_num_)
print_variables(row_num_df, "row_num_col")

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, row_number() OVER (ORDER BY id DESC) AS row_num_col from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  row_num_col
id                                                           
38     yes  2.65  Advanced    Beginner         1            3
36      no  3.00  Advanced      Novice         0            5
35      no  3.68    Novice    Beginner         1            6
34     yes  3.85  Advanced    Beginner         0            7
32     yes  3.46  Advanced    Beginner         0            9
31     yes  3.50  Advanced    Beginner         1           10
33      no  3.55    Novice      Novice         1            8
37      no  3.52    Novice      Novice         1            4
39     yes  3.75  Advanced    Beginner         0            2
40     yes  3.95    Novice    Beginner         0 

In [63]:
# Calculate cumulative distribution by id, partitioned over stats
row_num_ = func.row_number().over(partition_by=admissions_train.stats.expression, order_by=admissions_train.id.expression.desc())
type(row_num_)

sqlalchemy.sql.elements.Over

In [64]:
row_num_df = admissions_train.assign(row_num_col=row_num_)
print_variables(row_num_df, "row_num_col")

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, row_number() OVER (PARTITION BY stats ORDER BY id DESC) AS row_num_col from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  row_num_col
id                                                           
36      no  3.00  Advanced      Novice         0            3
32     yes  3.46  Advanced    Beginner         0            5
31     yes  3.50  Advanced    Beginner         1            6
30     yes  3.79  Advanced      Novice         0            7
27     yes  3.96  Advanced    Advanced         0            9
26     yes  3.57  Advanced    Advanced         1           10
8       no  3.60  Beginner    Advanced         1            1
6      yes  3.50  Beginner    Advanced         1            2
4      yes  3.50  Beginner      Novice         1            3
2      yes  3.76  Beginner    

### AVG function

In [65]:
### Let's get the mean gpa withing the window partitioned over stats and odered by id.
### SQL Clause Equivalent: "AVG(gpa) OVER(PARTION BY stats ORDER BY id)"
avggpa_part_ord = func.avg(admissions_train.gpa.expression).over(partition_by=admissions_train.stats.expression, 
                                                                 order_by=admissions_train.id.expression)
type(avggpa_part_ord)

sqlalchemy.sql.elements.Over

In [66]:
avg_df = admissions_train.assign(win_avg_gpa=avggpa_part_ord)
print_variables(avg_df, "win_avg_gpa")

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, avg(gpa) OVER (PARTITION BY stats ORDER BY id) AS win_avg_gpa from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  win_avg_gpa
id                                                           
11      no  3.13  Advanced    Advanced         1      3.50875
14     yes  3.45  Advanced    Advanced         0      3.50875
15     yes  4.00  Advanced    Advanced         1      3.50875
16      no  3.70  Advanced    Advanced         1      3.50875
18     yes  3.81  Advanced    Advanced         1      3.50875
19     yes  1.98  Advanced    Advanced         0      3.50875
1      yes  3.95  Beginner    Beginner         0      3.66200
2      yes  3.76  Beginner    Beginner         0      3.66200
4      yes  3.50  Beginner      Novice         1      3.66200
6      yes  3.50  Beginner    Advanced 

## Use cases for creating a window using ROWS BETWEEN syntax 

In [67]:
# Teradata offers various types of computations on windowed aggregate fucntions. Each computation 
#     1. Cumulative
#     2. Group
#     3. Moving
#     4. Remaining
# Each computation above can be performed using ROWS or ROWS BETWEEN syntax. Let's take a look at 
# examples of each type of computation.

# To perform windowed aggregate function over a window using ROWS and ROWS BETWEEN one must use 
# argument "rows" in over which accepts a tuple (p, f).
# Where,
#    p and f can accept Negative Integer Value, Positive Integer Value, 0 or None.
#    Each value passed to p and f have different meaning or results in different syntax.
#    In general, this what these values mean, i.e., SQL syntax will be generated based on these values.
#        1. Negative Value --> Indicates a preceding value
#        2. Positive Value --> Indiacates a following value
#        3. 0 --> For Current row
#        4. None --> Unbounded value.
#

### Windowed aggregation using 'Cumulative' Computation

In [68]:
## To perform "Cumulative" computation for a windowed aggregate fucntion, 
## one should use following SQL Syntax for such cases is:

#    1. ROWS UNBOUNDED PRECEDING ------------> No support available for this in SQLAlchemy. Needs more investigation.
#    2. ROWS BETWEEN UNBOUNDED PRECEDING AND value PRECEDING ---> rows=(None, negative value)
#    3. ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW ---> rows=(None, 0)
#    4. ROWS BETWEEN UNBOUNDED PRECEDING AND value FOLLOWING ---> rows=(None, positive value)

In [69]:
# Example to return Cumulative sum of gpa over a window of ROWS BETWEEN UNBOUNDED PRECEDING AND value PRECEDING
# ROWS BETWEEN UNBOUNDED PRECEDING AND value PRECEDING ---> rows=(None, negative value)
sum_gpa_rbupavp = func.sum(admissions_train.gpa.expression).over(order_by=admissions_train.stats.expression, rows=(None, -1))
df = admissions_train.assign(cumsum_gpa=sum_gpa_rbupavp)
print_variables(df)

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, sum(gpa) OVER (ORDER BY stats ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) AS cumsum_gpa from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  cumsum_gpa
id                                                          
15     yes  4.00  Advanced    Advanced         1        6.48
24      no  1.87  Advanced      Novice         1       14.38
25      no  3.96  Advanced    Advanced         1       16.25
16      no  3.70  Advanced    Advanced         1       20.21
34     yes  3.85  Advanced    Beginner         0       25.89
14     yes  3.45  Advanced    Advanced         0       29.74
19     yes  1.98  Advanced    Advanced         0       23.91
20     yes  3.90  Advanced    Advanced         1       10.48
38     yes  2.65  Advanced    Beginner         1        3.83
17      no  3.83  

In [70]:
# Example to return Cumulative sum of gpa over a window of Rows between ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
# ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW ---> rows=(None, 0)
sum_gpa_rbupavp = func.sum(admissions_train.gpa.expression).over(order_by=admissions_train.stats.expression, rows=(None, 0))
df = admissions_train.assign(sqlalchemy_gpa=sum_gpa_rbupavp)
print_variables(df)

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, sum(gpa) OVER (ORDER BY stats ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS sqlalchemy_gpa from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  sqlalchemy_gpa
id                                                              
13      no  4.00  Advanced      Novice         1            9.98
23     yes  3.59  Advanced      Novice         1           17.02
38     yes  2.65  Advanced    Beginner         1           19.67
36      no  3.00  Advanced      Novice         0           22.67
30     yes  3.79  Advanced      Novice         0           30.29
18     yes  3.81  Advanced    Advanced         1           34.10
17      no  3.83  Advanced    Advanced         1           26.50
14     yes  3.45  Advanced    Advanced         0           13.43
19     yes  1.98  Advanced    Advan

In [71]:
### ROWS BETWEEN UNBOUNDED PRECEDING AND value FOLLOWING ---> rows=(None, positive value)
sum_gpa_rbupavf = func.sum(admissions_train.gpa.expression).over(order_by=admissions_train.stats.expression, rows=(None, 2))
df = admissions_train.assign(sqlalchemy_gpa=sum_gpa_rbupavf)
print_variables(df)

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, sum(gpa) OVER (ORDER BY stats ROWS BETWEEN UNBOUNDED PRECEDING AND 2 FOLLOWING) AS sqlalchemy_gpa from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  sqlalchemy_gpa
id                                                              
15     yes  4.00  Advanced    Advanced         1           16.25
24      no  1.87  Advanced      Novice         1           23.91
25      no  3.96  Advanced    Advanced         1           25.89
16      no  3.70  Advanced    Advanced         1           29.74
34     yes  3.85  Advanced    Beginner         0           36.65
14     yes  3.45  Advanced    Advanced         0           40.47
19     yes  1.98  Advanced    Advanced         0           33.19
20     yes  3.90  Advanced    Advanced         1           20.21
38     yes  2.65  Advanced    Begin

### Windowed aggregation using 'Group' Computation

In [72]:
## Using "GROUP" computation.
## SQL Syntax for such cases is:
### ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING ---> rows=(None, None)
### OR using no "rows" argument at all.

In [73]:
### ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING ---> rows=(None, None)
sum_gpa_rbupauf = func.sum(admissions_train.gpa.expression).over(order_by=admissions_train.stats.expression, rows=(None, None))
df = admissions_train.assign(sqlalchemy_gpa=sum_gpa_rbupauf)
print_variables(df)

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, sum(gpa) OVER (ORDER BY stats ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS sqlalchemy_gpa from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  sqlalchemy_gpa
id                                                              
13      no  4.00  Advanced      Novice         1          141.67
23     yes  3.59  Advanced      Novice         1          141.67
38     yes  2.65  Advanced    Beginner         1          141.67
36      no  3.00  Advanced      Novice         0          141.67
30     yes  3.79  Advanced      Novice         0          141.67
18     yes  3.81  Advanced    Advanced         1          141.67
17      no  3.83  Advanced    Advanced         1          141.67
14     yes  3.45  Advanced    Advanced         0          141.67
19     yes  1.98  Advanced 

### Windowed aggregation using 'Moving' Computation

In [74]:
## Using "Moving" computation.
## SQL Syntax for such cases is:
### ROWS value PRECEDING ------------> No support available for this in SQLAlchemy.
### ROWS CURRENT ROW     ------------> No support available for this in SQLAlchemy.

#
# Values accepted by rows argument:
#      1. Negative Value --> Indicates a preceding value
#      2. Positive Value --> Indiacates a following value
#      3. 0 --> For Current row
#      4. None --> Unbounded value.


### ROWS BETWEEN value PRECEDING AND value PRECEDING ---> rows=(Negative Value, Negative Value)
### ROWS BETWEEN value PRECEDING AND CURRENT ROW ---> rows=(Negative Value, 0)
### ROWS BETWEEN value PRECEDING AND value FOLLOWING ---> rows=(Negative Value, Positive Value)
### ROWS BETWEEN CURRENT ROW AND CURRENT ROW ---> rows=(0, 0)
### ROWS BETWEEN CURRENT ROW AND value FOLLOWING ---> rows=(0, Positive Value)
### ROWS BETWEEN value FOLLOWING AND value FOLLOWING ---> rows=(Positive Value, Positive Value)

In [75]:
### ROWS BETWEEN value PRECEDING AND value PRECEDING ---> rows=(-3, -1)
sum_gpa_rbupauf = func.sum(admissions_train.gpa.expression).over(order_by=admissions_train.stats.expression, rows=(-3, -1))
df = admissions_train.assign(sqlalchemy_gpa=sum_gpa_rbupauf)
print_variables(df)

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, sum(gpa) OVER (ORDER BY stats ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING) AS sqlalchemy_gpa from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  sqlalchemy_gpa
id                                                              
15     yes  4.00  Advanced    Advanced         1            6.48
24      no  1.87  Advanced      Novice         1           10.55
25      no  3.96  Advanced    Advanced         1            9.77
16      no  3.70  Advanced    Advanced         1            9.73
34     yes  3.85  Advanced    Beginner         0            9.64
14     yes  3.45  Advanced    Advanced         0            9.53
19     yes  1.98  Advanced    Advanced         0            9.53
20     yes  3.90  Advanced    Advanced         1           10.48
38     yes  2.65  Advanced    Beginner     

In [76]:
### ROWS BETWEEN CURRENT ROW AND CURRENT ROW ---> rows=(0, 0)
sum_gpa_rbcr = func.sum(admissions_train.gpa.expression).over(order_by=admissions_train.stats.expression, rows=(0, 0))
df = admissions_train.assign(sqlalchemy_gpa=sum_gpa_rbcr)
print_variables(df)

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, sum(gpa) OVER (ORDER BY stats ROWS BETWEEN CURRENT ROW AND CURRENT ROW) AS sqlalchemy_gpa from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  sqlalchemy_gpa
id                                                              
13      no  4.00  Advanced      Novice         1            4.00
23     yes  3.59  Advanced      Novice         1            3.59
38     yes  2.65  Advanced    Beginner         1            2.65
36      no  3.00  Advanced      Novice         0            3.00
30     yes  3.79  Advanced      Novice         0            3.79
18     yes  3.81  Advanced    Advanced         1            3.81
17      no  3.83  Advanced    Advanced         1            3.83
14     yes  3.45  Advanced    Advanced         0            3.45
19     yes  1.98  Advanced    Advanced     

In [77]:
### ROWS BETWEEN CURRENT ROW AND 3 FOLLOWING ---> rows=(0, 3)
sum_gpa_rbcr3f = func.sum(admissions_train.gpa.expression).over(order_by=admissions_train.stats.expression, rows=(0, 3))
df = admissions_train.assign(sqlalchemy_gpa=sum_gpa_rbcr3f)
print_variables(df)

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, sum(gpa) OVER (ORDER BY stats ROWS BETWEEN CURRENT ROW AND 3 FOLLOWING) AS sqlalchemy_gpa from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  sqlalchemy_gpa
id                                                              
15     yes  4.00  Advanced    Advanced         1           13.73
24      no  1.87  Advanced      Novice         1           11.51
25      no  3.96  Advanced    Advanced         1           13.49
16      no  3.70  Advanced    Advanced         1           12.98
34     yes  3.85  Advanced    Beginner         0           14.58
14     yes  3.45  Advanced    Advanced         0           14.23
19     yes  1.98  Advanced    Advanced         0           12.74
20     yes  3.90  Advanced    Advanced         1           13.43
38     yes  2.65  Advanced    Beginner     

### Windowed aggregation using 'Remaining' Computation

In [78]:
## Using "REMAINING" computation.
## SQL Syntax for such cases is:

# Values accepted by rows argument:
#      1. Negative Value --> Indicates a preceding value
#      2. Positive Value --> Indiacates a following value
#      3. 0 --> For Current row
#      4. None --> Unbounded value.

### ROWS BETWEEN value PRECEDING AND UNBOUNDED FOLLOWING ---> rows=(Negative Value, None)
### ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING ---> rows=(0, None)
### ROWS BETWEEN value FOLLOWING AND UNBOUNDED FOLLOWING ---> rows=(Positive Value, None)

In [79]:
### ROWS BETWEEN 5 PRECEDING AND UNBOUNDED FOLLOWING ---> rows=(-5, None)
sum_gpa_rb5puf = func.sum(admissions_train.gpa.expression).over(order_by=admissions_train.stats.expression, rows=(-5, None))
df = admissions_train.assign(sqlalchemy_gpa=sum_gpa_rb5puf)
print_variables(df)

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, sum(gpa) OVER (ORDER BY stats ROWS BETWEEN 5 PRECEDING AND UNBOUNDED FOLLOWING) AS sqlalchemy_gpa from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa   stats programming  admitted  sqlalchemy_gpa
id                                                            
3       no  3.70  Novice    Beginner         1           27.68
21      no  3.87  Novice    Beginner         1           35.20
35      no  3.68  Novice    Beginner         1           39.15
33      no  3.55  Novice      Novice         1           43.10
37      no  3.52  Novice      Novice         1           50.46
29     yes  4.00  Novice    Beginner         0           53.96
22     yes  3.46  Novice    Beginner         0           46.70
12      no  3.65  Novice      Novice         1           31.20
7      yes  2.33  Novice      Novice         1         

In [80]:
### ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING ---> rows=(0, None)
sum_gpa_rbcruf = func.sum(admissions_train.gpa.expression).over(order_by=admissions_train.stats.expression, rows=(0, None))
df = admissions_train.assign(sqlalchemy_gpa=sum_gpa_rbcruf)
print_variables(df)

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, sum(gpa) OVER (ORDER BY stats ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS sqlalchemy_gpa from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa   stats programming  admitted  sqlalchemy_gpa
id                                                            
29     yes  4.00  Novice    Beginner         0           10.99
35      no  3.68  Novice    Beginner         1           18.37
37      no  3.52  Novice      Novice         1           21.89
40     yes  3.95  Novice    Beginner         0           25.84
12      no  3.65  Novice      Novice         1           31.82
22     yes  3.46  Novice    Beginner         0           35.28
7      yes  2.33  Novice      Novice         1           28.17
3       no  3.70  Novice    Beginner         1           14.69
33      no  3.55  Novice      Novice         1         

In [81]:
### ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING ---> rows=(0, None)
sum_gpa_rbcruf = func.AVG(admissions_train.admitted.expression).over(order_by=admissions_train.id.expression, rows=(0, None))
df = admissions_train.assign(sqlalchemy_gpa=sum_gpa_rbcruf)
print_variables(df)

Equivalent SQL: select id AS id, masters AS masters, gpa AS gpa, stats AS stats, programming AS programming, admitted AS admitted, AVG(admitted) OVER (ORDER BY id ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS sqlalchemy_gpa from "admissions_train"


 ************************* DataFrame ********************* 
   masters   gpa     stats programming  admitted  sqlalchemy_gpa
id                                                              
38     yes  2.65  Advanced    Beginner         1        0.333333
36      no  3.00  Advanced      Novice         0        0.400000
35      no  3.68    Novice    Beginner         1        0.500000
34     yes  3.85  Advanced    Beginner         0        0.428571
32     yes  3.46  Advanced    Beginner         0        0.444444
31     yes  3.50  Advanced    Beginner         1        0.500000
33      no  3.55    Novice      Novice         1        0.500000
37      no  3.52    Novice      Novice         1        0.500000
39     yes  3.75  Advanced    Beg

In [82]:
### Following Teradatda SQL syntaxes are not supported by SQLAlchemy.
# 1. RESET WHEN clause.
# 2. ROWS UNBOUNDED PRECEDING, ROWS value PRECEDING, ROWS CURRENT ROW

### SQLAlchemy offers range over in Window aggregate, Teradata does not support the same. 
# For example, "RANGE BETWEEN 5 PRECEDING AND 10 FOLLOWING" is not supported

In [83]:
# One must run remove_context() to close the connection and garbage collect internally generated objects.
remove_context()

True